In [33]:
# libraries
#!pip install rpy2
#!pip install pandas
#!pip install keras
#!pip install imblearn

    100% |████████████████████████████████| 92kB 660kB/s 
  Running setup.py bdist_wheel for imbalanced-learn ... - \ | / done
  Stored in directory: /Users/Kozodoi/Library/Caches/pip/wheels/b8/20/bd/0b775f7e5d413ac72562b1a5126598bcb6e0eae10da659be9f
Successfully built imbalanced-learn


In [1]:
# libraries
import rpy2.robjects as robjects
import pandas as pd
import keras
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import Adam
from sklearn import metrics

Using TensorFlow backend.


## 1. DATA PREPARATION

In [2]:
# load the data
path = "/Users/Kozodoi/Documents/Competitions/DSG_2017/"
data = pd.read_csv(path + "data/data_full.csv")
data.shape

(7578752, 35)

In [3]:
# add observation index
data["row_index"] = data.index

In [4]:
# data partitioning
tr = data.query("dataset == 'train'")
ts = data.query("dataset == 'test'")
kn = data.query("dataset != 'unknown'")
un = data.query("dataset == 'unknown'")

# print data sizes
print("train: "   + str(tr.shape))
print("test: "    + str(ts.shape))
print("known: "   + str(kn.shape))
print("unknown: " + str(un.shape))

train: (7519013, 36)
test: (39821, 36)
known: (7558834, 36)
unknown: (19918, 36)


In [5]:
# List numeric features used as predictors
print(data.columns)
numVars = ["user_ratio_flow", "user_ratio_full", "listen_type", "first_flow",
           "song_plays", "artist_plays", "platform_name1", "platform_name2",
           "song_skips", "artist_skips", "song_session_position", "time_diff"] 

# Create the data input matrix that can be passed to the keras model
tr_data = tr[[column for column in tr.columns if column in numVars]].as_matrix()
ts_data = ts[[column for column in ts.columns if column in numVars]].as_matrix()
kn_data = kn[[column for column in kn.columns if column in numVars]].as_matrix()
un_data = un[[column for column in un.columns if column in numVars]].as_matrix()

Index(['user_id', 'media_id', 'artist_id', 'genre_id', 'context_type',
       'media_duration', 'listen_type', 'user_gender', 'user_age',
       'is_listened', 'sample_id', 'dataset', 'session_id',
       'song_session_position', 'first_flow', 'time_diff', 'hours',
       'is_listened_lag', 'genre_plays', 'genre_skips', 'artist_plays',
       'artist_skips', 'album_plays', 'album_skips', 'song_plays',
       'song_skips', 'user_ratio_flow', 'user_ratio_full', 'genre_ratio',
       'artist_ratio', 'song_ratio', 'platform_name1', 'platform_name2',
       'platform_family1', 'platform_family2', 'row_index'],
      dtype='object')


## 2. MODELING

### 2.1. INITIALIZING

In [6]:
# Create an input layer with embeddings
user_in    = Input(shape = (1,), dtype = 'int64',   name = "user_in")
song_in    = Input(shape = (1,), dtype = 'int64',   name = "song_in")
artist_in  = Input(shape = (1,), dtype = 'int64',   name = "artist_in")
context_in = Input(shape = (1,), dtype = 'int64',   name = "context_in")

# Create an input layer with numeric features
data_in = Input(shape = (tr_data.shape[1],), name = "data_in")

# Counting number of unique ID values
n_users   = tr.user_id.nunique()
n_songs   = tr.media_id.nunique()
n_artists = tr.artist_id.nunique()
n_context = tr.context_type.nunique()

# Create an embedding assigning k latent factors to each ID
u = Embedding(n_users,   100, input_length = 1, embeddings_regularizer = l2(1e-5))(user_in)
s = Embedding(n_songs,   100, input_length = 1, embeddings_regularizer = l2(1e-5))(song_in)
a = Embedding(n_artists, 100, input_length = 1, embeddings_regularizer = l2(1e-5))(artist_in)
c = Embedding(n_context, 100, input_length = 1, embeddings_regularizer = l2(1e-5))(context_in)

# Layer with embeddings
embedding_input = concatenate([u, c])
embedding_input = Flatten()(embedding_input)
embedding_dense = Dense(128, activation = "relu")(embedding_input)
embedding_dense = BatchNormalization()(embedding_dense)

# Layer with numeric features
data_dense = Dense(16, activation = "relu")(data_in)
data_dense = BatchNormalization()(data_dense)

# Constructing the further layers
x = concatenate([embedding_dense, data_dense])
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x) 
x = Dropout(0.5)(x)
output = Dense(1, activation = "sigmoid")(x)

# Specify the model that we want to use
model = Model([user_in, context_in, data_in], output)
model.compile(optimizer = "Adagrad", loss = "binary_crossentropy", metrics = ['accuracy'])

### 2.2. FIRST STAGE

In [7]:
# run the estimations on training data
model.fit([tr.user_id, tr.context_type, tr_data], tr.is_listened, 
validation_data = ([ts.user_id, ts.context_type, ts_data], ts.is_listened),
batch_size = int(len(tr)/100), epochs = 10)

Train on 7519013 samples, validate on 39821 samples
Epoch 1/10
7519013/7519013 [==============================] - 175s - loss: 0.5530 - acc: 0.7343 - val_loss: 0.6707 - val_acc: 0.6076
Epoch 2/10
7519013/7519013 [==============================] - 165s - loss: 0.5129 - acc: 0.7574 - val_loss: 0.6615 - val_acc: 0.6078
Epoch 3/10
7519013/7519013 [==============================] - 180s - loss: 0.5049 - acc: 0.7607 - val_loss: 0.6457 - val_acc: 0.6141
Epoch 4/10
7519013/7519013 [==============================] - 174s - loss: 0.5005 - acc: 0.7624 - val_loss: 0.6364 - val_acc: 0.6450
Epoch 5/10
7519013/7519013 [==============================] - 292s - loss: 0.4970 - acc: 0.7640 - val_loss: 0.6186 - val_acc: 0.6757
Epoch 6/10
7519013/7519013 [==============================] - 275s - loss: 0.4948 - acc: 0.7650 - val_loss: 0.6103 - val_acc: 0.6790
Epoch 7/10
7519013/7519013 [==============================] - 274s - loss: 0.4934 - acc: 0.7653 - val_loss: 0.6176 - val_acc: 0.6764
Epoch 8/10
751901

In [8]:
# predict validation data
pred = pd.DataFrame()
pred["row_index"] = ts.row_index
pred["is_listened"] = model.predict([ts.user_id, ts.context_type, ts_data])
pred.to_csv(path + "pred_valid/keras_newdata_full.csv", index = False)
pred.head(5)

row_index  is_listened
5989        5989     0.990344
6053        6053     0.988549
6296        6296     0.988307
11840      11840     0.989002
11910      11910     0.990973

In [9]:
# computing AUC
metrics.roc_auc_score(ts.is_listened, pred.is_listened)

0.72155689722703342

### 2.3. SECOND STAGE

In [7]:
# run the estimations on full known data
model.fit([kn.user_id, kn.context_type, kn_data], kn.is_listened,
batch_size = int(kn.shape[0]/100), epochs = 10)

Epoch 1/10
7558834/7558834 [==============================] - 462s - loss: 0.5625 - acc: 0.7291   
Epoch 2/10
7558834/7558834 [==============================] - 435s - loss: 0.5160 - acc: 0.7561   
Epoch 3/10
7558834/7558834 [==============================] - 433s - loss: 0.5075 - acc: 0.7601   
Epoch 4/10
7558834/7558834 [==============================] - 427s - loss: 0.5027 - acc: 0.7619   
Epoch 5/10
7558834/7558834 [==============================] - 410s - loss: 0.4992 - acc: 0.7633   
Epoch 6/10
7558834/7558834 [==============================] - 283s - loss: 0.4967 - acc: 0.7641   
Epoch 7/10
7558834/7558834 [==============================] - 281s - loss: 0.4946 - acc: 0.7650   
Epoch 8/10
7558834/7558834 [==============================] - 189s - loss: 0.4932 - acc: 0.7653   
Epoch 9/10
7558834/7558834 [==============================] - 180s - loss: 0.4917 - acc: 0.7659   
Epoch 10/10
7558834/7558834 [==============================] - 259s - loss: 0.4906 - acc: 0.7662   


In [8]:
# predict unknown data
pred = pd.DataFrame()
pred["sample_id"] = un.sample_id.astype(int)
pred["is_listened"] = model.predict([un.user_id, un.context_type, un_data])
pred = pred.sort_values("sample_id")
pred.to_csv(path + "pred_unknown/keras_newdata_full_context100.csv", index = False)
pred.head(5)

sample_id  is_listened
7551765          0     0.936919
6913499          1     0.566037
6529339          2     0.708621
5409309          3     0.428296
6218646          4     0.842011